In [ ]:
import time
import numpy as np
#进行随机选择
import random
import matplotlib.pyplot as plt
import random
import os

In [ ]:
gapx, gapy = 20, 8
W1, W2 = 3, 1
minX, maxX = -20, 500
minY, maxY = -20, 500
point=[]
layout=[]
sourcefolder = "./DenseMap/DenseMapReadyToRun/"
savefolder = "./resultSA2.0/"

In [ ]:
def prepare_data(selectlist, filename):
    point.clear()
    layout.clear()
    num = 1
    for line in open(filename):
        index, x, y = line.strip().split(" ")
        if int(index) in selectlist:
            point.extend([(num,(float(x),float(y)))])
            num = num + 1
    for i in range(len(point)):
        layout.append([i+1, random.randint(1,4), 0])


In [ ]:
"""
Get the tagrec bottom-left
"""
def getPos(i,m):
    pointx = point[i][1][0]
    pointy = point[i][1][1]
    # print(pointx, pointy)
    if m==1:
        return [(pointx,pointy),(pointx+gapx,pointy),(pointx+gapx,pointy+gapy),(pointx,pointy+gapy)]
    elif m==2:
        return [(pointx-gapx,pointy),(pointx,pointy),(pointx,pointy+gapy),(pointx-gapx,pointy+gapy)]
    elif m==3:
        return [(pointx-gapx,pointy-gapy),(pointx,pointy-gapy),(pointx,pointy),(pointx-gapx,pointy)]
    else:
        return [(pointx, pointy-gapy),(pointx+gapx,pointy-gapy),(pointx+gapx,pointy),(pointx,pointy)]

"""
to test is it inside of a rec
"""
def inner(point, rec):
    if point[0] > rec[0][0] and point[1] > rec[0][1] and point[0] < rec[1][0] and point[1] > rec[1][1] and point[0] < rec[2][0] and point[1] < rec[2][1] and point[0] > rec[3][0] and point[1] < rec[3][1]:
        return True
    return False

"""
判断冲突
vect:layout list
"""
def conflict(vect,i,j):
    m = vect[i][1]
    n = vect[j][1]
    reci = getPos(i,m)
    recj = getPos(j,n)
    for i in range(len(reci)):
        if inner(reci[i], recj):
            return True
    return False

"""
count each point its confilict count
"""
def costconfilct(vect):
    sum = 0
    for i in range(len(point)):
        temp = 0
        for j in range(len(point)):
            if i != j:
                if(conflict(vect,i,j)):
                    temp = temp + 1
        vect[i][2] = temp
        sum = sum + temp
    # print("conflict:"+str(sum))
    return sum

# print(costroad(layout))

In [ ]:
"""
each point its layout cost
"""
def cost_function(layout):
    res = 0
    for i in range(len(point)):
        res = res + layout[i][1]*0.1
    # print(res)
    return res
print(cost_function(layout))

"""
计算每个点total cost
"""
def costroad(vect):
    conflict = costconfilct(vect)
    conflict = W1*conflict + W2*cost_function(vect)
    return conflict
costroad(layout)

In [ ]:
"""
lower compute complexity version of total cost
used in tabu search update
"""
def costrod_lite(new_set, old_set, pointx):
    conflict_sum = 0
    for i in range(len(point)):
        a = conflict(new_set, i, pointx-1)
        b = conflict(old_set, i, pointx-1)
        # print(i,pointx,a,b)
        if a is True and b is False:
            # conflict_sum = conflict_sum + 1
            new_set[i][2] = new_set[i][2]+1
        elif a is False and b is True:
            # conflict_sum = conflict_sum - 1
            new_set[i][2] = new_set[i][2]-1
    # print(new_set[pointx-1][1], old_set[pointx-1][1])
    for i in range(len(point)):
        conflict_sum = conflict_sum + new_set[i][2]
    # print("lite")
    # print(conflict_sum)
    conflict_sum = 3*conflict_sum + cost_function(new_set) - new_set[pointx-1][1]*0.1 + old_set[pointx-1][1]*0.1
    return conflict_sum

In [ ]:
def cost_dif(new_set, old_set, pointx):
    conflict_sum = 0
    for i in range(len(point)):
        a = conflict(new_set, i, pointx-1)
        b = conflict(old_set, i, pointx-1)
        # print(i,pointx,a,b)
        if a is True and b is False:
            conflict_sum = conflict_sum + 1
        elif a is False and b is True:
            conflict_sum = conflict_sum - 1
    # print(new_set[pointx-1][1], old_set[pointx-1][1])
    conflict_sum = conflict_sum * 3 - new_set[pointx-1][1]*0.1 + old_set[pointx-1][1]*0.1
    return conflict_sum
            

In [ ]:
import copy
def generate_new_set(old_set):
    sel = random.sample(old_set,1)
    while True:
        nextPos = random.randint(1,4)
        if nextPos != sel[0][1]:
            break
    m = [sel[0][0],sel[0][1],nextPos]
    new_set = copy.deepcopy(old_set)
    new_set[sel[0][0]-1][1] = m[2]
    diff = cost_dif(new_set, old_set, m[0])
    # print(old_set)
    return (new_set,m,diff)
# a,b,c = generate_new_set(layout)
# print(c)

In [ ]:
"""
to plot
"""
import matplotlib.pyplot as plt
def plot(layout,name):
        def create_rect(i, width, height):
                pointx = point[i][1][0]
                pointy = point[i][1][1]
                m = layout[i][1]
                if m==1:
                        x,y = pointx,pointy
                elif m==2:
                        x, y = pointx-width,pointy
                elif m==3:
                        x, y = pointx-width,pointy-height
                else:
                        x, y = pointx, pointy-height
                rect=plt.Rectangle(
                        (x, y),  # (x,y)矩形左下角
                        width,  # width长
                        height,  # height宽
                        # color='maroon', 
                        fill = False)
                return rect
        fig1 = plt.figure(figsize=(16,9))
        plt.axis('off')
        ax1 = fig1.add_subplot(111)

        for i in range(len(point)):
                ax1.add_patch(create_rect(i, gapx, gapy))
        # plt.xlim(2400,5200)
        # plt.ylim(7100,12812)
        # plt.xlim(-20,500)
        # plt.ylim(-20,500)
        plt.xlim(minX,maxX)
        plt.ylim(minY,maxY)
        plt.title("{}".format(name))
        # plt.savefig("./result/resultImg-{}.svg".format(name), format="svg")
        plt.show()
# plot()
# plt.savefig('scatter.eps',dpi=600,format='eps')

def save(layout, cost_result, time, path):
        #gapx, gapy = 20, 8
        # W1, W2 = 3, 1
        # minX, maxX = -20, 500
        # minY, maxY = -20, 500
        f = open(path, "w")
        f.write("Process Time: {}\n".format(time))
        f.write("cost_init,cost_final,confilct_init,conflict_final\n")
        for i in range(len(cost_result)):
                if i == len(cost_result)-1:
                        f.write("{}\n".format(cost_result[i]))
                else:
                        f.write("{} ".format(cost_result[i]))
        f.write("gapx, gapy = {},{}\n".format(gapx, gapy))
        f.write("minX, maxX = {},{}\n".format(minX, maxX))
        f.write("minY, maxY = {},{}\n".format(minY, maxY))
        for i in range(len(layout)):
                towrite = str(point[i][1][0]) + " " + str(point[i][1][1]) + " " + str(layout[i][1]) + "\n"
                f.write(towrite)
        f.close()

In [ ]:
def sasearch(origin_times, init_temperature, min_temperature, cooling_rate, iterates, namefortest):
    # init T
    T = init_temperature
    no_cnt = 0
    layout = []
    result_layout = []
    result_record = []
    for t1 in range(origin_times):
        layout.clear()
        for i in range(len(point)):
            layout.append([i+1, random.randint(1,4), 0])
        print("initial state:")
        cost_init = costroad(layout)
        confilct_init = costconfilct(layout)
        print(cost_init)
        nameprefix = str(namefortest) + str(cost_init)
        plot(layout=layout,name=nameprefix)
        # plot(layout=layout)
        # print(cost_function(layout))
        while T > min_temperature:
            old_cost = cost_function(layout)
            result_record.append(old_cost)
            for i in range(iterates):
                # old_cost = cost_function(layout)
                # result_record.append(old_cost)
                result_layout.append(layout)
                new_set, m, diff = generate_new_set(old_set=layout)
                # print("diff:" + str(diff))
                # diff = new_cost - old_cost
                if diff < 0:
                    # 重新计数
                    no_cnt = 0
                    # 采纳新解
                    layout = new_set[:]
                else:
                    no_cnt = no_cnt + 1
                    # 新解更坏，这时候以一定几率踩纳
                    # np.exp(-(valuenew-valuecurrent)/t)
                    P = np.exp(-(diff)/T)
                    # print("P:"+str(P))
                    # 如果随机数在概率之内，则采纳
                    if random.random() < P:
                        layout = new_set[:]
            # 退火降温
            T = T * cooling_rate
            print(T)
            # 超过5000次没有被采纳，则结束迭代
            if no_cnt > 5000:
                break
    return (result_record, result_layout, layout, cost_init, confilct_init)

In [ ]:
filelist = []
for root, dirs, files in os.walk(sourcefolder):  
    # filelist = dirs.sort()
    filelist = sorted(files)
print(len(filelist))

In [ ]:
# li = [220,250,280,310,340]
li = [130,160,190]
# li = range(0,100,10)
for i in li:
    filename = os.path.join(sourcefolder, filelist[i])
    selectlist = range(1,10000)
    prepare_data(selectlist, filename)
    print("{} {} started".format(i,filename))
    # a: layout d:cost
    tic = time.time()
    result_record, result_layout, layout, cost_init, confilct_init = sasearch(origin_times=1,init_temperature=200, min_temperature=1e-2, cooling_rate=0.90, iterates=500, namefortest=filelist[i])
    toc = time.time()
    processTime = toc-tic
    cost_final = costroad(layout)
    confilct_final = costconfilct(layout)
    cost_result = []
    cost_result.append(cost_init)
    cost_result.append(cost_final)
    cost_result.append(confilct_init)
    cost_result.append(confilct_final)
    print(cost_final, confilct_final)
    plot(layout=layout,name=filelist[i])
    savefilename = os.path.join(savefolder, filelist[i])
    save(layout, cost_result, processTime, savefilename)
    print("{} finished".format(i))